In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ----------------------------------------------------------------------
# STEP 1: SETUP AND DATA LOADING (FIXED DTYPEWARNING)
# ----------------------------------------------------------------------

# 1. Load the Raw Datasets
try:
    # FIX for DtypeWarning: Added low_memory=False to suppress the warning
    historical_data_df = pd.read_csv('historical_data.csv', low_memory=False)
    fear_greed_df = pd.read_csv('fear_greed_index.csv')
    print("Successfully loaded both datasets.")
except FileNotFoundError as e:
    print(f"Error: One or both files not found. Please ensure both CSVs are uploaded to the Colab environment. {e}")
    exit()

# Create the required output directories (as per assignment rules)
os.makedirs('csv_files', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
print("Created 'csv_files/' and 'outputs/' directories.")


# ----------------------------------------------------------------------
# STEP 2: DATA CLEANING, AGGREGATION, AND INTEGRATION
# ----------------------------------------------------------------------

# --- A. Prepare Historical Trader Data ---

# FIX for ValueError: Use 'errors='coerce'' to handle malformed timestamps
historical_data_df['Timestamp_DT'] = pd.to_datetime(
    historical_data_df['Timestamp IST'],
    errors='coerce', # Turns bad entries (like the one that failed) into NaT
    dayfirst=True   # Ensures correct DD-MM-YYYY interpretation
)

# Drop rows where the timestamp could not be parsed (NaT values)
historical_data_df.dropna(subset=['Timestamp_DT'], inplace=True)

# Extract the date only
historical_data_df['Date'] = historical_data_df['Timestamp_DT'].dt.date

# Aggregate daily trading metrics
daily_trader_data_df = historical_data_df.groupby('Date').agg(
    Total_PnL=('Closed PnL', 'sum'),
    Total_Volume_USD=('Size USD', 'sum'),
    Unique_Traders=('Account', 'nunique'),
    Total_Trades=('Order ID', 'count')
).reset_index()

# Convert 'Date' column back to datetime objects for merging
daily_trader_data_df['Date'] = pd.to_datetime(daily_trader_data_df['Date'])
print("\nHistorical Trader Data aggregated daily.")


# --- B. Prepare Fear & Greed Index Data ---
fear_greed_df['Date'] = pd.to_datetime(fear_greed_df['date'])

# Standardize sentiment classification
def standardize_sentiment(classification):
    if 'Fear' in classification:
        return 'Fear'
    elif 'Greed' in classification:
        return 'Greed'
    else: # Neutral
        return 'Neutral'

fear_greed_df['Sentiment'] = fear_greed_df['classification'].apply(standardize_sentiment)


# --- C. Merge the Datasets ---
sentiment_data = fear_greed_df[['Date', 'value', 'Sentiment']].rename(columns={'value': 'FGI_Value'})
merged_df = pd.merge(daily_trader_data_df, sentiment_data, on='Date', how='inner')

# Save the intermediate merged file to the required directory
merged_df.to_csv('csv_files/merged_trader_sentiment_data.csv', index=False)
print("Datasets merged and saved to 'csv_files/merged_trader_sentiment_data.csv'.")


# ----------------------------------------------------------------------
# STEP 3: CORE ANALYSIS AND INSIGHT GENERATION
# ----------------------------------------------------------------------

# 1. Group Analysis: Compare Mean/Total Metrics by Sentiment
sentiment_analysis = merged_df.groupby('Sentiment').agg(
    Mean_Daily_PnL=('Total_PnL', 'mean'),
    Total_PnL_Sum=('Total_PnL', 'sum'),
    Mean_Daily_Volume_USD=('Total_Volume_USD', 'mean'),
    Total_Volume_USD_Sum=('Total_Volume_USD', 'sum'),
    Mean_Unique_Traders=('Unique_Traders', 'mean'),
    Total_Days=('Sentiment', 'count')
).sort_values(by='Total_PnL_Sum', ascending=False)

print("\n--- Summary of Trading Metrics by Sentiment ---")
print(sentiment_analysis.to_markdown(numalign="left", stralign="left"))


# 2. Visualization: Function to create and save charts
def create_bar_chart(data, column, title, filename, ylabel):
    plt.figure(figsize=(9, 6))
    plot_data = data.sort_values(by=column, ascending=False)
    color_map = {'Greed': 'green', 'Fear': 'red', 'Neutral': 'gray'}
    colors = [color_map.get(s, 'blue') for s in plot_data.index]
    bars = plt.bar(plot_data.index, plot_data[column], color=colors)
    plt.title(title, fontsize=14)
    plt.xlabel('Market Sentiment', fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.xticks(rotation=0, ha='center')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    for bar in bars:
        yval = bar.get_height()
        if abs(yval) > 1e6:
            text = f'${yval/1e6:,.2f}M'
        elif abs(yval) > 1e3:
            text = f'${yval/1e3:,.1f}K'
        else:
            text = f'${yval:,.0f}'

        # Adjust text placement for visual clarity
        plt.text(bar.get_x() + bar.get_width()/2, yval + (yval * 0.05) if yval > 0 else yval - (abs(yval) * 0.05),
                 text, ha='center', va='bottom' if yval > 0 else 'top', fontsize=10, weight='bold')

    plt.tight_layout()
    plt.savefig(f'outputs/{filename}')
    plt.close()


# 3. Generate Charts

# Mean Daily PnL (Highest Profitability Signal)
create_bar_chart(sentiment_analysis, 'Mean_Daily_PnL',
                 'Mean Daily PnL by Market Sentiment (Profitability)',
                 'mean_daily_pnl_by_sentiment.png',
                 'Mean Daily PnL (USD)')

# Total Volume (Activity Signal)
create_bar_chart(sentiment_analysis, 'Total_Volume_USD_Sum',
                 'Total Aggregate Volume by Market Sentiment (Activity)',
                 'total_volume_by_sentiment.png',
                 'Total Volume (USD)')

print("\nAll steps completed successfully. Download the files from the 'csv_files/' and 'outputs/' folders for your final submission!")

Successfully loaded both datasets.
Created 'csv_files/' and 'outputs/' directories.

Historical Trader Data aggregated daily.
Datasets merged and saved to 'csv_files/merged_trader_sentiment_data.csv'.

--- Summary of Trading Metrics by Sentiment ---
| Sentiment   | Mean_Daily_PnL   | Total_PnL_Sum   | Mean_Daily_Volume_USD   | Total_Volume_USD_Sum   | Mean_Unique_Traders   | Total_Days   |
|:------------|:-----------------|:----------------|:------------------------|:-----------------------|:----------------------|:-------------|
| Greed       | 11120.5          | 3.30278e+06     | 1.04784e+06             | 3.1121e+08             | 2.14815               | 297          |
| Fear        | 30074.1          | 3.06756e+06     | 3.62923e+06             | 3.70181e+08            | 3.36275               | 102          |
| Neutral     | 17097.2          | 1.14551e+06     | 1.90622e+06             | 1.27717e+08            | 2.77612               | 67           |

All steps completed successfully. 